In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from  langchain_google_genai import ChatGoogleGenerativeAI
from langchain_ollama.llms import OllamaLLM

google_llm = ChatGoogleGenerativeAI(model = "gemini-2.5-flash")
gemma_llm = OllamaLLM(model="gemma3:4b")


In [5]:
target_url = "https://www.headphonezone.in/pages/our-team"
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(target_url)
doc = loader.load()

from langchain_text_splitters import RecursiveCharacterTextSplitter

# Clean up whitespace
cleaned_doc = []
for d in doc:
    d.page_content = d.page_content.replace("\n", " ").replace("\t", " ")
    d.page_content = " ".join(d.page_content.split())  # remove extra spaces
    cleaned_doc.append(d)

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

texts = text_splitter.split_documents(cleaned_doc)
len(texts)


from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OllamaEmbeddings(
    model="mxbai-embed-large:latest",
)


vectorstore = FAISS.from_documents(texts, embedding=embeddings)

In [14]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import  ChatPromptTemplate
from langchain.chains import create_retrieval_chain


prompt = ChatPromptTemplate.from_template(
    """
    Answer the question based on the context below.

    Question: {input}

    <content>
    {context}
    </content>

    If you don't know the answer, just say "I don't know".
    """,
)


document_chain = create_stuff_documents_chain(gemma_llm, prompt)

retriver = vectorstore.as_retriever(search_kwargs={"k": 2})

retrival_chain = create_retrieval_chain(retriver, document_chain)

In [18]:
retrival_chain.invoke({"input": "who is FOUNDER & CEO of headphone zone"})

{'input': 'who is FOUNDER & CEO of headphone zone',
 'context': [Document(id='da076f74-6a16-4cbc-b92f-817543613050', metadata={'source': 'https://www.headphonezone.in/pages/our-team', 'title': 'Meet The Team Of Headphone Zone', 'description': 'Headphone Zone is India’s First Exclusive Headphone Retail Brand. Know more about our team and what everyone here at Headphone Zone does.', 'language': 'en'}, page_content='Meet The Team Of Headphone Zone Skip to content Festival of Sound Sale is Now Live 💥 Festival of Sound Sale is Now Live 💥 Festival of Sound Sale is Now Live 💥 Festival of Sound Sale is Now Live 💥 Festival of Sound Sale is Now Live 💥 Festival of Sound Sale is Now Live 💥 Festival of Sound Sale is Now Live 💥 Festival of Sound Sale is Now Live 💥 Festival of Sound Sale is Now Live 💥 Festival of Sound Sale is Now Live 💥 Menu Search SALE Categories In-Ears Headphones DACs & Amps Hi-Res Audio Players Wireless Cables Accessories Home Audio Merch In-Ears Beginner Audiophile IEMs Flagshi